In [2]:
import pandas as pd
import altair as alt
import eco_style
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [35]:
image = alt.Chart(pd.DataFrame([{'url': 'https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/refs/heads/main/guidelines/logos/eco-logo-light.png'}])).mark_image(
    height=30,
    width=90,
    opacity=0.5,
    # align='right',
    baseline='bottom'
).encode(
    url='url',
    x=alt.value(400),
    y=alt.value(0)
)

# Chart: UK top Exports to US

In [41]:
df = pd.read_excel("data/news_TradeData.xlsx", sheet_name="Top20")

rebinds = {
  "Road vehicles (including air-cushion vehicles)": "Road Vehicles",
  "Medicinal and pharmaceutical products": "Medicinal and Pharmaceutical",
  "Power-generating machinery and equipment": "Power-Generating Equipment",
  "Miscellaneous manufactured articles, n.e.s.": "Misc. Manufactured Articles",
  "General industrial machinery and equipment, n.e.s., and machine parts, n.e.s.": "Industrial Machinery and Parts",
  "Professional, scientific and controlling instruments and apparatus, n.e.s.": "Scientific and Control Instruments",
  "Electrical machinery, apparatus and appliances, n.e.s., and electrical parts thereof (including non-electrical counterparts, n.e.s., of electrical household-type equipment)": "Electrical Machinery and Parts",
  "Other transport equipment": "Other Transport Equipment",
  "Machinery specialized for particular industries": "Specialised Industrial Machinery",
  "Gold, non-monetary (excluding gold ores and concentrates)": "Gold (Non-Monetary)",
  "Non-ferrous metals": "Non-Ferrous Metals",
  "Beverages": "Beverages",
  "Organic chemicals": "Organic Chemicals",
  "Special transactions and commodities not classified according to kind": "Special Transactions and Commodities",
  "Chemical materials and products, n.e.s.": "Chemical Materials",
  "Manufactures of metals, n.e.s.": "Metal Manufactures",
  "Telecommunications and sound-recording and reproducing apparatus and equipment": "Telecom and Audio Equipment",
  "Petroleum, petroleum products and related materials": "Petroleum and Related Products",
  "Furniture and parts thereof; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings": "Furniture and Bedding",
  "Office machines and automatic data-processing machines": "Office and Data Machines"
}

df['shortProduct'] = df['Type'].replace(rebinds)
df['Value'] = df['Value'].astype(float)/1000000000

df['order'] = df['Value'].rank(method='first', ascending=False)

base = alt.Chart(df).encode(
    x=alt.X('Value:Q', title='', axis=alt.Axis(format="$,.0f", labelExpr="datum.label + 'B'")),
    y=alt.Y('shortProduct:N', title='', sort="-x"),
    order=alt.Order('order:Q', sort='ascending'),
).properties(
    width=400,
    height=500
)

labels = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=12
).encode(
    x=alt.value(0),
    y=alt.Y('shortProduct:N', title=''),
    text=alt.Text('shortProduct:N'),
)

bars = base.mark_bar(color="#36b7b4")

chart = bars + image.encode(x=alt.value(335), y=alt.value(-10)) 

chart


/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.LayerChart(...)

# Chart: Modes of Transport

In [54]:
df = pd.read_excel("data/news_TradeData.xlsx", sheet_name="Transport")
df.columns = ['mode', 'value']

df['value'] = df['value'].astype(float)/1000000000

df['share'] = df['value'] / df['value'].sum()
df = df.iloc[:3]
other_share = 1-df.iloc[:2]['share'].sum()
other_share
df.iloc[2, 2] = other_share
df



base = alt.Chart(df).encode(
    x=alt.X('value:Q', title='', axis=alt.Axis(format="$,.0f", labelExpr="datum.label + 'B'")),
    y=alt.Y('mode:N', title='', sort="-x"),
).properties(
    width=400,
    height=200
)

labels = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=12
).encode(
    x=alt.value(0),
    y=alt.Y('mode:N', title=''),
    text=alt.Text('mode:N'),
)

bars = base.mark_bar(color="#36b7b4")

bars + labels

chart = bars + image.encode(x=alt.value(335), y=alt.value(-10))

chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)